In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from subprocess import check_output
print(check_output(["ls", "../MNIST_data/fashion_csv/"]).decode("utf8"))

fashion-mnist_test.csv
fashion-mnist_train.csv.tar.gz



In [2]:
from keras.utils import to_categorical
import gzip
import pandas as pd
import os


# 完整的csv格式的训练集太大, 无法上传至github, 因此保存压缩文件, 在此解压缩并写入同名csv.
train_file = gzip.GzipFile("../MNIST_data/fashion_csv/fashion-mnist_train.csv.tar.gz", "r")
with open("../MNIST_data/fashion_csv/fashion-mnist_train.csv", 'wb') as f:
    f.write(train_file.read())
data_train = pd.read_csv('../MNIST_data/fashion_csv/fashion-mnist_train.csv')
data_train = data_train.drop(index=len(data_train.iloc[:, 0])-1) # 不知为什么, 最后一行全为nan, 因此删去末行.
os.remove('../MNIST_data/fashion_csv/fashion-mnist_train.csv')
data_test = pd.read_csv('../MNIST_data/fashion_csv/fashion-mnist_test.csv')

# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets("../MNIST_data/fashion/", one_hot=True)
# X_train, y_train = mnist.train.images, mnist.train.labels
# X_test, y_test = mnist.test.images, mnist.test.labels

Using TensorFlow backend.


In [3]:
X_train = np.array(data_train.iloc[:, 1:])
X_test = np.array(data_test.iloc[:, 1:])
y_train = to_categorical(np.array(data_train.iloc[:, 0]))
y_test = to_categorical(np.array(data_test.iloc[:, 0]))


img_rows, img_cols = 28, 28
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D

input_shape = (img_rows, img_cols, 1)
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.compile(optimizer='adam',
          loss='categorical_crossentropy',
          metrics=['accuracy'])

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1600)              0         
__________

In [8]:
model.fit(X_train, y_train,
          batch_size=50,
          epochs=60,
          verbose=1,
          validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)
model.save_weights('my_model_weights.h5')

Train on 60000 samples, validate on 10000 samples
Epoch 1/60
60000/60000 [==============================] - 47s 787us/step - loss: 0.5862 - acc: 0.7856 - val_loss: 0.3691 - val_acc: 0.8655
Epoch 2/60
60000/60000 [==============================] - 34s 565us/step - loss: 0.4003 - acc: 0.8543 - val_loss: 0.3127 - val_acc: 0.8838
Epoch 3/60
60000/60000 [==============================] - 34s 568us/step - loss: 0.3548 - acc: 0.8717 - val_loss: 0.2858 - val_acc: 0.8930
Epoch 4/60
60000/60000 [==============================] - 34s 570us/step - loss: 0.3243 - acc: 0.8814 - val_loss: 0.2560 - val_acc: 0.9062
Epoch 5/60
60000/60000 [==============================] - 35s 577us/step - loss: 0.3031 - acc: 0.8896 - val_loss: 0.2717 - val_acc: 0.8947
Epoch 6/60
60000/60000 [==============================] - 35s 575us/step - loss: 0.2910 - acc: 0.8940 - val_loss: 0.2484 - val_acc: 0.9077
Epoch 7/60
60000/60000 [==============================] - 34s 575us/step - loss: 0.2795 - acc: 0.8960 - val_loss: 0.

60000/60000 [==============================] - 40s 662us/step - loss: 0.1662 - acc: 0.9366 - val_loss: 0.2015 - val_acc: 0.9290
Epoch 60/60
60000/60000 [==============================] - 37s 624us/step - loss: 0.1640 - acc: 0.9388 - val_loss: 0.2102 - val_acc: 0.9288


In [16]:
print('accuracy: ',score[1])
print('loss: ',score[0])

accuracy:  0.9288
loss:  0.21016990533


In [10]:
print(score)

[0.21016990533024071, 0.92879999999999996]


In [11]:
model.save("weights_input.h5")
from keras.models import load_model
model_l = load_model(r"./weights_input.h5")
import read_image
data = read_image.Dataset()   

(6500, 28, 28, 1)
共6500张图片


In [17]:
prediction = model.predict(data.images)

In [18]:
with open("PengZheng.txt", "w") as f:
    for i in range(prediction.shape[0]):
        f.write(str(np.argmax(prediction[i])))
        if i != 6499:
            f.write("\n")

In [19]:
res = []
with open("PengZheng.txt", "r") as f:
    for i in range(6500):
        res.append(int(f.readline()))
labels = []
with open("../../../test_label.txt", "r") as f:
    for i in range(6500):
        labels.append(int(f.readline()))

In [20]:
import numpy as np
sum(np.array(res) == np.array(labels)) / len(res)

0.78476923076923077